In [1]:
# %% Imports
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax
import pandas as pd
import numpy as np

# %% Load dataset and create train-test sets
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Extract features and target
X_train = train_data.iloc[:, :-1].to_numpy()  # Ensure conversion to NumPy array
y_train = train_data.iloc[:, -1].to_numpy()   # Target from training data
X_test = test_data.iloc[:, :-1].to_numpy()    # Features from test data
y_test = test_data.iloc[:, -1].to_numpy()     # Target from test data

In [2]:
# Define variable names
var_names = ["Feature_" + str(i) for i in range(X_train.shape[1])]

# Define the hyperparameter grid
param_grid = {
    'nr_clus': [5, 10, 15],  # Number of clusters
    'clustering_method': ['fcm', 'gk']  # Clustering methods
}

# Perform grid search
best_params = None
best_score = 0
best_metrics = {}

for nr_clus in param_grid['nr_clus']:
    for method in param_grid['clustering_method']:
        print(f"Testing parameters: nr_clus={nr_clus}, method={method}")
        
        # %% Cluster the input-output space
        cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=nr_clus)
        clust_centers, part_matrix, _ = cl.cluster(method=method)
        
        # %% Estimate membership functions parameters
        ae = AntecedentEstimator(X_train, part_matrix)
        antecedent_params = ae.determineMF()
        
        # %% Estimate consequent parameters
        ce = ConsequentEstimator(X_train, y_train, part_matrix)
        conseq_params = ce.suglms()
        
        # %% Build first-order Takagi-Sugeno model
        modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, var_names, save_simpful_code=False)
        model = modbuilder.get_model()
        
        # %% Get model predictions
        modtester = SugenoFISTester(model, X_test, var_names)
        y_pred_probs = clip(modtester.predict()[0], 0, 1)
        y_pred_probs = column_stack((1 - y_pred_probs, y_pred_probs))
        y_pred = argmax(y_pred_probs, axis=1)
        
        # %% Compute classification metrics
        acc_score = accuracy_score(y_test, y_pred)
        rec_score = recall_score(y_test, y_pred)
        prec_score = precision_score(y_test, y_pred)
        F1_score = f1_score(y_test, y_pred)
        kappa = cohen_kappa_score(y_test, y_pred)
        
        print(f"Results: Accuracy={acc_score:.3f}, Precision={prec_score:.3f}, Recall={rec_score:.3f}, "
              f"F1={F1_score:.3f}, Kappa={kappa:.3f}")
        
        # Update best parameters if this configuration is better
        if acc_score > best_score:
            best_score = acc_score
            best_params = {'nr_clus': nr_clus, 'method': method}
            best_metrics = {
                'accuracy': acc_score,
                'precision': prec_score,
                'recall': rec_score,
                'f1': F1_score,
                'kappa': kappa
            }

# Output the best parameters and their metrics
print("\nBest Hyperparameters and Metrics:")
print(f"Best Parameters: {best_params}")
print(f"Metrics: {best_metrics}")

Testing parameters: nr_clus=5, method=fcm
 * Detected 5 rules / clusters
 * Detected Sugeno model type
Results: Accuracy=0.892, Precision=0.872, Recall=0.918, F1=0.894, Kappa=0.783
Testing parameters: nr_clus=5, method=gk
 * Detected 5 rules / clusters
 * Detected Sugeno model type
Results: Accuracy=0.892, Precision=0.872, Recall=0.918, F1=0.894, Kappa=0.783
Testing parameters: nr_clus=10, method=fcm
 * Detected 10 rules / clusters
 * Detected Sugeno model type
Results: Accuracy=0.892, Precision=0.872, Recall=0.918, F1=0.894, Kappa=0.783
Testing parameters: nr_clus=10, method=gk
 * Detected 10 rules / clusters
 * Detected Sugeno model type
Results: Accuracy=0.892, Precision=0.872, Recall=0.918, F1=0.894, Kappa=0.783
Testing parameters: nr_clus=15, method=fcm
 * Detected 15 rules / clusters
 * Detected Sugeno model type
Results: Accuracy=0.892, Precision=0.872, Recall=0.918, F1=0.894, Kappa=0.783
Testing parameters: nr_clus=15, method=gk
 * Detected 15 rules / clusters
 * Detected Sugen